In [3]:
import yfinance as yf
import pandas as pd
import numpy as np

# 1. DEFINE THE UNIVERSE (Example: Top Industrial Stocks)
tickers = ['UNP', 'HON', 'GE', 'CAT', 'DE', 'MMM', 'ETN', 'ITW', 'EMR', 'PH', 'AAPL','CROX']

def get_buffett_metrics(ticker):
    try:
        stock = yf.Ticker(ticker)

        # Financials
        balance_sheet = stock.balance_sheet
        income_stmt = stock.income_stmt
        cash_flow = stock.cashflow

        # Modified ROIC
        ebit = income_stmt.loc['EBIT'].iloc[0]
        invested_capital = balance_sheet.loc['Stockholders Equity'].iloc[0] + \
                           balance_sheet.loc['Total Debt'].iloc[0]
        roic = ebit / invested_capital

        # Net Debt / EBITDA
        total_debt = balance_sheet.loc['Total Debt'].iloc[0]
        cash = balance_sheet.loc['Cash And Cash Equivalents'].iloc[0]
        net_debt = total_debt - cash
        ebitda = income_stmt.loc['EBITDA'].iloc[0]
        leverage = net_debt / ebitda

        # Owner valuation
        ocf = cash_flow.loc['Operating Cash Flow'].iloc[0]
        capex = cash_flow.loc['Capital Expenditure'].iloc[0] # Usually negative
        fcf = ocf + capex # Adding negative number
        market_cap = stock.info['marketCap']
        fcf_yield = fcf / market_cap

        return {
            'Ticker': ticker,
            'Price': stock.info['currentPrice'],
            'ROIC (%)': round(roic * 100, 2),
            'Net Debt/EBITDA': round(leverage, 2),
            'FCF Yield (%)': round(fcf_yield * 100, 2),
            'P/E Ratio': round(stock.info['trailingPE'], 2)
        }

    except Exception as e:
        return None

data = []
print(f"Scanning {len(tickers)} companies...")
for t in tickers:
    metrics = get_buffett_metrics(t)
    if metrics:
        data.append(metrics)

df = pd.DataFrame(data)

# Apply filter
quality_stocks = df[
    (df['ROIC (%)'] > 12) &
    (df['Net Debt/EBITDA'] < 3) &
    (df['FCF Yield (%)'] > 4)
].sort_values(by='ROIC (%)', ascending=False)

print("--- THE QUALITY LIST ---")
print(quality_stocks)


Scanning 12 companies...
--- THE QUALITY LIST ---
   Ticker    Price  ROIC (%)  Net Debt/EBITDA  FCF Yield (%)  P/E Ratio
11   CROX   90.150     28.83             1.40          18.75      31.30
0     UNP  234.695     20.39             2.52           4.23      19.92
